# Goodreads books recommender system

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gc
from ast import literal_eval
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import math
from scipy.stats import pearsonr


In [2]:
books_df = pd.read_csv('books.csv',converters={"genres": literal_eval})
ratings_df = pd.read_csv('ratings.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

## Books

In [ ]:
genres = set(books_df["genres"].explode())
genres

In [ ]:
books_df.info()

In [ ]:
# most rated books
books_df.sort_values('ratings_count', ascending = False)[["title","authors","average_rating", "ratings_count"]].iloc[0:20]

In [ ]:
# authors of top 100 rated books
top_100 = books_df.sort_values(['average_rating'], ascending = False).iloc[0:100,:]

# transform authors to list and get top 15 authors that appear in top 100 books
authors_top_100 = pd.Series([x for item in top_100['authors'] for x in item]).value_counts().to_frame('counts').reset_index()
authors_top_100 = authors_top_100.iloc[0:15,:]

In [ ]:
sns.set_style('darkgrid')
sns.set_palette('cividis')

In [ ]:
ax = sns.histplot(data=books_df, x="average_rating", bins=20)
ax.set_title("Distribution of Book Ratings")
ax.set_xlabel("Average Rating")
ax.set_ylabel("Number of books")
plt.show()

In [ ]:
temp_books = pd.DataFrame()
temp_books["ratings_count_log"]=np.log(books_df.ratings_count)
ax = sns.histplot(data = temp_books, x = "ratings_count_log")
ax.set_title("Distribution of Book Ratings Count")
ax.set_xlabel("Log of number of ratings")
ax.set_ylabel("NUmber of books")
sns.despine()
plt.show()

In [ ]:
ax = sns.scatterplot(data = books_df, x = "ratings_count", y = "average_rating")
ax.set(xlim=(0, 1000000))
ax.set_title("Scatterplot of average book rating vs number of ratings")
ax.set_xlabel("Number of ratings")
ax.set_ylabel("Rating")
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{(x/1000000)}M'))
sns.regplot(data=books_df, x="ratings_count", y="average_rating", scatter=False, color='r')

In [ ]:
columns_to_plot = ['average_rating', 'ratings_count']
fig, axes = plt.subplots(ncols=len(columns_to_plot))
for column, axis in zip(columns_to_plot, axes):
    if column == 'ratings_count':
        sns.boxplot(data=books_df[column], ax=axis).set(ylim=(0, 100000))
        axis.set_title('Number of ratings')
    else:
        sns.boxplot(data=books_df[column], ax=axis)
        axis.set_title('Average rating')
plt.tight_layout()
plt.show()

## Ratings

In [ ]:
ratings_df.head()

In [ ]:
ratings_df.info()

In [ ]:
len(ratings_df["book_id"].unique())

In [ ]:
ratings_df[["rating"]].describe()

In [ ]:
ratings_df.groupby('user_id')['book_id'].count().head()

In [ ]:
ax = sns.boxplot(y=ratings_df.groupby('user_id')['book_id'].count(), orient="v")
ax.set_ylim(0, 100)
ax.set_ylabel("Number of ratings")
ax.set_title("Number of ratings by user")

## Naive Baseline

In [8]:
import math

mean = train.groupby('book_id').mean()
id_score_dict = mean['rating'].to_dict()

sum = 0
n = 0
for index, row in test.iterrows():
    id = row['book_id']
    sum += (row['rating'] - id_score_dict[id])**2
    n += 1

math.sqrt(sum/n)

1.1846826946244475

RMSE=1.184

## Collaborative filtering

We can use the p-value to determine better the similarity between the users.

[4,4], [4,4] has larger similarity than [4,4,4,3,3,4,5], [4,5,4,3,3,4,5], but the second is more preferable.

It is slow as it can be, and i does not work well (:

In [4]:
# dictionary from books to list of users which rated the given book
book_user_dict = {}
grouped_data = train.groupby('book_id')
for book_id, group in grouped_data:
    user_generator = (x for x in group['user_id'])
    book_user_dict[book_id] = list(user_generator)
    
# dictionary from users to dictionary of books to rating given by the user
user_book_rating_dict = {}
grouped_data = train.groupby('user_id')
for user_id, group in grouped_data:
    book_rating_dict = {book_id: rating for book_id, rating in zip(group['book_id'], group['rating'])}
    user_book_rating_dict[user_id] = book_rating_dict

In [112]:
number_of_books_required = 10
k_neighbours = 10
corr_threshold = 0.1
r2 = False

def user_similarity(r1, r2):
    vals1, vals2 = [], []
    for key in r1:
        if key in r2:
            vals1.append(r1[key])
            vals2.append(r2[key])
    if len(vals1) < number_of_books_required:
        return None
    corr, _= pearsonr(vals1, vals2)
    if corr < corr_threshold:
        return None
    return corr

def predict(user_id, book_id):
    similarities = []
    for user_id2 in book_user_dict[book_id]:
        similarity = user_similarity(user_book_rating_dict[user_id], user_book_rating_dict[user_id2])
        if similarity == None or np.isnan(similarity):
            continue
        similarities.append((similarity, user_book_rating_dict[user_id2][book_id]))
    if len(similarities) < k_neighbours:
        return id_score_dict[book_id]
    similarities = sorted(similarities, key=lambda x: x[0])
    summ = 0
    weights = 0
    for i in range(k_neighbours):
        sim = similarities[i][0]
        if r2:
            sim = sim * sim
        summ += similarities[i][1] * sim
        weights += sim
    
    return summ / weights

In [114]:
%%time
n = 0 # n is a counter so I can stop it early
s = 0
for _, row in test.iterrows():
    predicted = predict(row['user_id'], row['book_id'])
    s += (row['rating']-predicted)  ** 2
    n += 1
    if n % 10000 == 0:
        print(math.sqrt(s/n))
math.sqrt(s/n)

1.1844659834328997
1.2209770534282907
1.1966747158474076
1.2007794280863233
1.2033019700471452
1.1993663294899979
1.1964774317255416
1.2043393867054888
1.210265230214759
1.2097654493151695
1.2013639814443906
1.1998265940697048
1.202164435333428
1.2004460176013012
1.2064433997049357
1.206302719429371
1.2084119910072084
1.2089400343113494
1.2099390027278794
1.213548300731373
1.2156494331408008
1.217626946623452
1.2160935324104336
1.2167258603734332
1.2188832348413967
1.2184750631289205
1.2215819970956714
1.2226889951509234
1.2205524217156556
1.2197440276908804
1.2189051284317745
1.217233600292917
1.215883034203377
1.2147890519063143
1.2153738472563924
1.214020555094872
1.2126514947189044
1.212299836653791
1.2117762612853296
1.2108433556385745
1.2106922105983249
1.2096367330017785
1.2085548787604556
1.2073138469111673
1.2064667346274727
1.205592972268254
1.2048633515573899
1.205355606553403
1.205070454051272
1.2037280811190518
1.2032045980404822
1.2023025249279307
1.2019755598778357
1.201

1.1994205491356822

number_of_books_required = 10
k_neighbours = 10
corr_threshold = 0.5
r2 = False

1.1885425924759414

number_of_books_required = 10
k_neighbours = 10
corr_threshold = 0.5
r2 = True

1.1889582857761327

number_of_books_required = 10
k_neighbours = 10
corr_threshold = 0.1
r2 = False

1.1994205491356822

In [ ]:
len(ratings_df["user_id"].unique())

In [43]:
len(test)

635993